In [8]:
!pip install openai

In [9]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
import pdb

In [10]:
client = OpenAI(
    api_key = "sk-******************************************************",
)

In [11]:
question = "What is the way to approach a professor as a prospective graduate student"
question

'What is the way to approach a professor as a prospective graduate student'

In [14]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{"role": "user", "content": question}])

response

ChatCompletion(id='chatcmpl-9N7c9TuwgQVHzpbIdgAIadLM9RUwR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="When approaching a professor as a prospective graduate student, it is important to be professional and respectful. Here are some steps to follow:\n\n1. Do your research: Before reaching out to the professor, make sure you have a clear understanding of their research interests, publications, and any recent projects they have been involved in. This will show the professor that you have taken the time to familiarize yourself with their work.\n\n2. Write a personalized email: When contacting the professor, make sure to address them by their title (e.g. Dr., Professor) and use proper grammar and spelling in your email. Be sure to briefly introduce yourself, explain your background and interests, and mention why you are interested in working with them specifically.\n\n3. Be clear about your intentions: Be upfront about your interest 

In [15]:
response.choices[0].message.content

"When approaching a professor as a prospective graduate student, it is important to be professional and respectful. Here are some steps to follow:\n\n1. Do your research: Before reaching out to the professor, make sure you have a clear understanding of their research interests, publications, and any recent projects they have been involved in. This will show the professor that you have taken the time to familiarize yourself with their work.\n\n2. Write a personalized email: When contacting the professor, make sure to address them by their title (e.g. Dr., Professor) and use proper grammar and spelling in your email. Be sure to briefly introduce yourself, explain your background and interests, and mention why you are interested in working with them specifically.\n\n3. Be clear about your intentions: Be upfront about your interest in their research and express your desire to work with them as a graduate student. Mention any specific skills or experiences you have that you believe would be

# Implementing RAG(Retrieval Augmented Generation)

In [17]:
df = pd.read_csv("/kaggle/input/university-prof/profile_data.csv")
df.shape

(31, 4)

In [20]:
df = df.dropna()

In [21]:
df.shape

(29, 4)

In [22]:
modified_rows  = []
for index, row in df.iterrows():
    # Add "is a" after the name
    modified_name = row['Name'] + ' is a '
    # Concatenate the university and text
    concatenated_text = row['Varsity'] + 'professor. He has research interest in differnt fields. Such as ' + row['Research']
    # Concatenate the modified name and concatenated text
    modified_rows.append(modified_name + concatenated_text)

# Create a new column with the concatenated values
df['Text'] = modified_rows
    

In [23]:
def get_embedding(text, model = 'text-embedding-3-small'):
    text = text.replace(","," ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
get_embedding(df["Research"].iloc[0])

In [27]:
%%time

df["Research_emb_Text"] = df["Text"].apply(get_embedding)

CPU times: user 320 ms, sys: 44.5 ms, total: 365 ms
Wall time: 8min 48s


In [28]:
df.to_csv("research_profile.csv", index=False)
df.to_pickle("research_profile.pkl")

In [29]:
%%time
df = pd.read_pickle("/kaggle/working/research_profile.pkl")
df

CPU times: user 3.47 ms, sys: 100 µs, total: 3.57 ms
Wall time: 3.13 ms


,Name,Email,Research,Varsity,Text,Research_emb_Text
0,Dr. Hongchi Shi,hs15@txstate.edu,"Parallel and distributed computing, Wireless s...",Texas State University,Dr. Hongchi Shi is a Texas State Universitypro...,"[-0.009071694687008858, -0.06133105233311653, ..."
1,Dr. Moonis Ali,ma04@txstate.edu,"Artificial intelligence, knowledge-based exper...",Texas State University,Dr. Moonis Ali is a Texas State Universityprof...,"[-0.006536086555570364, -0.027764108031988144,..."
2,Dr. Tanzima Islam,tanzima@txstate.edu,"Performance, scalability, fault-tolerance, mac...",Texas State University,Dr. Tanzima Islam is a Texas State Universityp...,"[-0.030250024050474167, -0.03261330723762512, ..."
3,Dr. Randall Klepetko,sox16@txstate.edu,"Security, AI, ML, Image / Video, and Acoustics.",Texas State University,Dr. Randall Klepetko is a Texas State Universi...,"[-0.0561802014708519, -0.014636171981692314, 0..."
4,Dr. Oleg V Komogortsev,ok@txstate.edu,"Cybersecurity (biometrics), human computer int...",Texas State University,Dr. Oleg V Komogortsev is a Texas State Univer...,"[-0.007154195569455624, -0.01955736242234707, ..."
5,Dr. Ziliang Zong,ziliang@txstate.edu,"Energy-Efficient Computing and Systems, Green ...",Texas State University,Dr. Ziliang Zong is a Texas State Universitypr...,"[0.02133183553814888, -0.024705875664949417, 0..."
6,Dr. Kecheng Yang,yangk@txstate.edu,"Real-time Systems, Scheduling, and Resource Al...",Texas State University,Dr. Kecheng Yang is a Texas State Universitypr...,"[-0.03463056683540344, -0.02248615026473999, 0..."
7,Dr. Jelena Tesic,jtesic@txstate.edu,"Machine Learning, Visual Recognition, Deep Neu...",Texas State University,Dr. Jelena Tesic is a Texas State Universitypr...,"[-0.050947584211826324, -0.047271400690078735,..."
8,Dr. Jill Seaman,jillseaman@txstate.edu,"BA, Messiah College, Grantham, PA; PhD, Penn S...",Texas State University,Dr. Jill Seaman is a Texas State Universitypro...,"[-0.021614456549286842, -0.04727783426642418, ..."
9,Dr. Heena Rathore,heena.rathore@txstate.edu,"cognitive AI, cyber physical systems security,...",Texas State University,Dr. Heena Rathore is a Texas State Universityp...,"[0.01802137866616249, -0.04420700669288635, 0...."


In [30]:
prompt = "Find me a professor who has research interest in High Performance Computing"
prompt


'Find me a professor who has research interest in High Performance Computing'

In [31]:
prompt_embedding = get_embedding(prompt)
prompt, prompt_embedding[0:10], "....."

('Find me a professor who has research interest in High Performance Computing',
 [-0.03892303258180618,
  -0.035195328295230865,
  0.03335568308830261,
  0.016254238784313202,
  0.0018910832004621625,
  -0.01048113964498043,
  -0.05552825704216957,
  0.001400158042088151,
  0.02416955679655075,
  -0.04320747032761574],
 '.....')

In [32]:
def fn(page):
    return np.dot(page, prompt_embedding)


df["distance"] = df["Research_emb_Text"].apply(fn)

In [37]:
df.sort_values('distance', ascending=False,inplace=True)

df

,Name,Email,Research,Varsity,Text,Research_emb_Text,distance
11,Dr. Apan Qasem,apan@txstate.edu,"Compilers, Architecture, GPU Computing, Machin...",Texas State University,Dr. Apan Qasem is a Texas State Universityprof...,"[-0.012466241605579853, -0.058884404599666595,...",0.506546
29,Dr. Martin Burtscher,burtscher@txstate.edu,"GPU computing, program parallelization, graph ...",Texas State University,Dr. Martin Burtscher is a Texas State Universi...,"[-0.03387865796685219, -0.03442429006099701, 0...",0.505628
2,Dr. Tanzima Islam,tanzima@txstate.edu,"Performance, scalability, fault-tolerance, mac...",Texas State University,Dr. Tanzima Islam is a Texas State Universityp...,"[-0.030250024050474167, -0.03261330723762512, ...",0.477091
23,Dr. Tanzima Islam,tanzima@txstate.edu,"Performance, scalability, fault-tolerance, mac...",Texas State University,Dr. Tanzima Islam is a Texas State Universityp...,"[-0.030250024050474167, -0.03261330723762512, ...",0.477091
0,Dr. Hongchi Shi,hs15@txstate.edu,"Parallel and distributed computing, Wireless s...",Texas State University,Dr. Hongchi Shi is a Texas State Universitypro...,"[-0.009071694687008858, -0.06133105233311653, ...",0.439232
19,Dr. Chul-Ho Lee,chulho.lee@txstate.edu,"Data Science, Network Science, Networking, and...",Texas State University,Dr. Chul-Ho Lee is a Texas State Universitypro...,"[-0.037607889622449875, -0.058149274438619614,...",0.438618
24,Dr. C Jinshong Hwang,cjhwang@txstate.edu,"Database systems, object oriented systems, sof...",Texas State University,Dr. C Jinshong Hwang is a Texas State Universi...,"[-0.03604786470532417, -0.06355807930231094, 0...",0.435023
6,Dr. Kecheng Yang,yangk@txstate.edu,"Real-time Systems, Scheduling, and Resource Al...",Texas State University,Dr. Kecheng Yang is a Texas State Universitypr...,"[-0.03463056683540344, -0.02248615026473999, 0...",0.433759
5,Dr. Ziliang Zong,ziliang@txstate.edu,"Energy-Efficient Computing and Systems, Green ...",Texas State University,Dr. Ziliang Zong is a Texas State Universitypr...,"[0.02133183553814888, -0.024705875664949417, 0...",0.429518
13,Dr. Wuxu Peng,wuxu@txstate.edu,"Cloud computing, computer communications and s...",Texas State University,Dr. Wuxu Peng is a Texas State Universityprofe...,"[-0.012122582644224167, -0.061543773859739304,...",0.428955


In [36]:
context = df["Text"].iloc[0] + "\n" + df["Text"].iloc[1] + "\n" + df["Text"].iloc[2] + "\n" + df["Text"].iloc[3] + "\n" + df["Text"].iloc[4] + "\n" + df["Text"].iloc[5]

print(context)

Dr. Apan Qasem is a Texas State Universityprofessor. He has research interest in differnt fields. Such as Compilers, Architecture, GPU Computing, Machine Learning
Dr. Martin Burtscher is a Texas State Universityprofessor. He has research interest in differnt fields. Such as GPU computing, program parallelization, graph processing, algorithm synthesis, data compression
Dr. Tanzima Islam is a Texas State Universityprofessor. He has research interest in differnt fields. Such as Performance, scalability, fault-tolerance, machine learning, high performance computing systems
Dr. Tanzima Islam is a Texas State Universityprofessor. He has research interest in differnt fields. Such as Performance, scalability, fault-tolerance, machine learning, high performance computing systems
Dr. Hongchi Shi is a Texas State Universityprofessor. He has research interest in differnt fields. Such as Parallel and distributed computing, Wireless sensor networks, Image processing, and Neural networks
Dr. Chul-Ho 

In [40]:
responses = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [
                {"role": "system", "content": "You are an assistant who is helping to find professor for specific field. When answering the question use the third context to answer the question."},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": f"Use this information while answering question from university webpage from user question.{context}. Please stick to this context while answeing question."}
            ])

In [42]:
responses.choices[0].message.content

'Dr. Tanzima Islam is a Texas State University professor who has research interest in High Performance Computing.'

In [43]:
question = "Is there any professor who is good at HPC?"

In [44]:
responses = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [
                {"role": "system", "content": "You are an assistant who is helping to find professor for specific field. When answering the question use the third context to answer the question."},
                {"role": "user", "content": question},
                {"role": "assistant", "content": f"Use this information while answering question from university webpage from user question.{context}. Please stick to this context while answeing question."}
            ])

In [45]:
responses.choices[0].message.content

'Yes, Dr. Tanzima Islam at Texas State University is a professor known for his expertise in High Performance Computing (HPC). His research interests include performance, scalability, fault-tolerance, machine learning, and high performance computing systems.'

In [46]:
question2 = "Is there any other professor who is good at HPC?"

In [47]:
responses = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [
                {"role": "system", "content": "You are an assistant who is helping to find professor for specific field. When answering the question use the third context to answer the question."},
                {"role": "user", "content": question2},
                {"role": "assistant", "content": f"Use this information while answering question from university webpage from user question.{context}. Please stick to this context while answeing question."}
            ])

In [48]:
responses.choices[0].message.content

'Yes, Dr. Apan Qasem, Dr. Martin Burtscher, Dr. Tanzima Islam, Dr. Hongchi Shi, and Dr. Chul-Ho Lee are other professors at Texas State University who excel in the field of High Performance Computing (HPC).'

In [49]:
info = "please tell me one of the professor email who is good at HPC?"

In [50]:
responses = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [
                {"role": "system", "content": "You are an assistant who is helping to find professor for specific field. When answering the question use the third context to answer the question."},
                {"role": "user", "content": info},
                {"role": "assistant", "content": f"Use this information while answering question from university webpage from user question.{context}. Please stick to this context while answeing question."}
            ])

In [51]:
responses.choices[0].message.content

'One of the professors who is good at High Performance Computing (HPC) at Texas State University is Dr. Tanzima Islam. You can reach out to Dr. Islam at t_i2@txstate.edu for more information on HPC research and opportunities.'